
# PTNT Sanity Check · Research-Grade Tutorial

**Goal.** Produce a *provable* sanity check, per supervisor request:

1. Build a **synthetic oracle** (data generator).
2. Run fresh **hold-out experiments** through both:
   - the **oracle** and
   - the **learned parameterized model** (LPDO).
3. Compare outputs on **validation** with formal metrics:
   - cross-entropy vs **data-entropy** floors,
   - **per-circuit Hellinger fidelity** distributions.
4. Declare **PASS/FAIL** with explicit criteria.

We implement two scenarios:

- **Baseline**: depolarizing + coherent over-rotation, **no** env memory.
- **Memory**: nonzero `rzz` env coupling (temporal correlations).



In [15]:

# Make a nearby PTNT checkout importable if not pip-installed.
import os, sys, pathlib
roots = [pathlib.Path.cwd(), *pathlib.Path.cwd().parents]
for r in roots[:4]:
    if (r / "ptnt").is_dir() and str(r) not in sys.path:
        sys.path.insert(0, str(r))

# Basic environment info
try:
    import ptnt
    from ptnt._version import __version__ as ptnt_version
    print("[ptnt] import OK → version:", ptnt_version)
except Exception as e:
    print("[ptnt] import failed:", e)
    raise

try:
    import jax
    print("[ptnt] JAX devices:", jax.devices())
except Exception as e:
    print("[ptnt] JAX not available:", e)


[ptnt] import OK → version: 0.1.0
[ptnt] JAX devices: [CpuDevice(id=0)]



## Quimb compatibility patch (safe fallback)

Some `quimb` versions expect `row_tag_id/col_tag_id`; others want `y_tag_id/x_tag_id` in `TensorNetwork2DFlat.from_TN`.
We patch the internal helper used by the likelihood so the tutorial runs reliably across versions.




- Quimb changed arg names across versions (row_tag_id/col_tag_id vs y_tag_id/x_tag_id). The patch tries new names first, then falls back. This prevents the “we need to specify x_tag_id” error I saw.

- We Ensure the operator array -> 2D TN conversion always works, regardless of quimb version.

- Sanity mapping: This is infrastructure so the TN contracts; it only makes the tutorial robust

In [16]:

# Runtime patch for op_arrays_to_single_vector_TN_padded to support both quimb call styles
import types
import quimb.tensor as qtn
import ptnt.preprocess.shadow as sh

def _op_arrays_to_single_vector_TN_padded_robust(op_seq):
    k = len(op_seq[0]) - 1
    nQ = op_seq.shape[0]
    TN_list = []
    for i in range(nQ):
        initial = qtn.Tensor(
            op_seq[i][0], inds=(f"kP_q{i}", f"ko{k}_q{i}"),
            tags=["U3", f"q{i}_U{k}", f"ROW{i}", f"COL{k}"],
        )
        for j, O in enumerate(op_seq[i][1:]):
            initial = initial & qtn.Tensor(
                O,
                inds=(f"ki{k-j}_q{i}", f"ko{k-j-1}_q{i}"),
                tags=["U3", f"q{i}_U{k-j-1}", f"ROW{i}", f"COL{k-j-1}"],
            )
        TN_list.append(initial)

    OTN_ket = qtn.TensorNetwork(TN_list)
    try:
        OTN_ket = qtn.tensor_2d.TensorNetwork2DFlat.from_TN(
            OTN_ket,
            site_tag_id="q{}_U{}",
            Ly=k + 1, Lx=nQ,
            row_tag_id="ROW{}",
            col_tag_id="COL{}",
        )
    except Exception:
        OTN_ket = qtn.tensor_2d.TensorNetwork2DFlat.from_TN(
            OTN_ket,
            site_tag_id="q{}_U{}",
            Ly=k + 1, Lx=nQ,
            y_tag_id="ROW{}",
            x_tag_id="COL{}",
        )

    OTN_bra = OTN_ket.H.copy()
    OTN_bra.reindex_({f"ko{i}_q{j}": f"bo{i}_q{j}" for i in range(k + 1) for j in range(nQ)})
    OTN_bra.reindex_({f"ki{i}_q{j}": f"bi{i}_q{j}" for i in range(1, k + 1) for j in range(nQ)})
    OTN_ket.add_tag("OP KET"); OTN_bra.add_tag("OP BRA")
    return OTN_ket & OTN_bra

# Inject the patch
sh.op_arrays_to_single_vector_TN_padded = _op_arrays_to_single_vector_TN_padded_robust
print("Patched: ptnt.preprocess.shadow.op_arrays_to_single_vector_TN_padded")


Patched: ptnt.preprocess.shadow.op_arrays_to_single_vector_TN_padded



## Scenario selection

- **Baseline**: no env memory; oracle = Aer with depolarizing + coherent over-rotation on `sx`.
- **Memory**: env coupling `rzz=0.20`; oracle = Aer without extra static noise.

We keep small sizes so the check is quick; scale them up for stronger signal.



**Imports:** Aer for simulation; our circuit builders; shadow preprocessing tables; Full-U converters.

**SCENARIO switch:** sets Q,T and the oracle dynamics:
- Baseline: env_IA = 0, noise_model = depol + coherent Rx on sx.
- Memory: env_IA.rzz = 0.2, noise_model = None (we isolate temporal coupling).

**Shell:** base_PT_circ_template(..., "dd_clifford") builds the randomized-compiling shell with env ancilla on wire 0.

**Sanity mapping:** “synthetic oracle” defined for both baseline and memory. we decide which underlying physics generates the data.

In [11]:

from qiskit_aer import Aer
import numpy as np, quimb as qu
import matplotlib.pyplot as plt

from ptnt.circuits.templates import base_PT_circ_template
from ptnt.circuits.noise_models import create_env_IA, make_coherent_depol_noise_model
from ptnt.circuits.utils import bind_ordered

from ptnt.preprocess.shadow import (
    clifford_param_dict, validation_param_dict, shadow_results_to_data_vec,
    shadow_seqs_to_op_array,
    clifford_measurements_vT, clifford_unitaries_vT,
    val_measurements_vT, val_unitaries_vT,
)

SCENARIO = "baseline"   # "baseline" or "memory"

if SCENARIO == "baseline":
    Q, T = 2, 2
    env = create_env_IA(0.0, 0.0, 0.0)
    noise_model = make_coherent_depol_noise_model(0.001, 0.02)
    shots_char, shots_val = 1024, 4096
    N_train, N_val = 240, 80
else:
    Q, T = 2, 3
    env = create_env_IA(0.0, 0.0, 0.20)
    noise_model = None
    shots_char, shots_val = 1024, 8192
    N_train, N_val = 300, 100

backend = Aer.get_backend("aer_simulator")
shell = base_PT_circ_template(Q, T, backend, basis_gates=None, template="dd_clifford", env_IA=env)

print("Scenario:", SCENARIO, "| Q,T =", (Q, T))
print(shell)


Scenario: baseline | Q,T = (2, 2)
              ┌─────────┐          ┌──────────┐                              »
q_0: ─────────┤ Ry(π/4) ├──────────┤0         ├──────────────────────────────»
     ┌────────┴─────────┴─────────┐│  Unitary │┌────────────────────────────┐»
q_1: ┤ U(t0_q0_x,t0_q0_y,t0_q0_z) ├┤1         ├┤ U(t1_q0_x,t1_q0_y,t1_q0_z) ├»
     ├────────────────────────────┤└──────────┘└────────────────────────────┘»
q_2: ┤ U(t0_q1_x,t0_q1_y,t0_q1_z) ├──────────────────────────────────────────»
     └────────────────────────────┘                                          »
c: 2/════════════════════════════════════════════════════════════════════════»
                                                                             »
«     ┌──────────┐         ┌──────────┐                                       »
«q_0: ┤0         ├─────────┤0         ├───────────────────────────────────────»
«     │          │         │  Unitary │         ┌────────────────────────────┐»
«q_1: ┤  Unitar

## Generate & simulate shadows (training + validation)


build_batch: samples random Clifford indices for training and U3 for validation; uses bind_ordered to assign angles to the shell’s parameters reliably.

Simulate with the oracle: Aer generates counts; training shots lower than validation (so val entropy floor is tighter).

shadow_results_to_data_vec: counts -> probabilities with endianness corrected (consistent bit order).

entropy_floor: computes  the empirical entropy of the validation distribution , the best possible CE any model can reach on that data.

Sanity mapping: “run new experiments (fresh circuits), compute oracle distributions, and record the shot-noise floor.”

In [4]:

def reverse_seq_list(seq_list):
    out = []
    for seq in seq_list:
        tmp = []
        for Tseq in seq:
            tmp.append([o for o in reversed(Tseq)])
        tmp.reverse()
        out.append(tmp)
    return out

def build_batch(template, N, table, T, Q):
    circs, seqs = [], []
    for _ in range(N):
        idx = np.random.randint(0, len(table), size=(T+1, Q))
        seqs.append(idx.T)
        params = np.array([table[i] for i in idx.ravel()])
        circs.append(bind_ordered(template, params.ravel()))
    return circs, seqs

def entropy_floor(prob_vec):
    v = np.array(prob_vec, dtype=float)
    v[v < 1e-12] = 1e-12
    return - (1/len(v)) * v @ np.log(v + 1e-18)

# Generate and simulate
train_circs, train_seqs = build_batch(shell, N_train, clifford_param_dict, T, Q)
val_circs,   val_seqs   = build_batch(shell, N_val,   validation_param_dict, T, Q)

job_t = backend.run(train_circs, shots=shots_char, noise_model=noise_model)
job_v = backend.run(val_circs,   shots=shots_val,   noise_model=noise_model)

train_counts = job_t.result().get_counts()
val_counts   = job_v.result().get_counts()

train_vec, train_keys = shadow_results_to_data_vec(train_counts, shots_char, Q)
val_vec,   val_keys   = shadow_results_to_data_vec(val_counts,   shots_val,   Q)

print("train / val items:", len(train_vec), len(val_vec))
print("data-entropy (train / val):", entropy_floor(train_vec), entropy_floor(val_vec))


train / val items: 791 319
data-entropy (train / val): 0.2844237016147668 0.23593343154709287


## Sequences -> operator arrays (Full-U view)

“convert sequences to the objects the likelihood contracts with” (so oracle and model interface match).

In [5]:

# Convert sequences -> operator arrays (Full-U view)
train_full = shadow_seqs_to_op_array(reverse_seq_list(train_seqs), train_keys, clifford_measurements_vT, clifford_unitaries_vT)
val_full   = shadow_seqs_to_op_array(reverse_seq_list(val_seqs),   val_keys,   val_measurements_vT,   val_unitaries_vT)

print("Full-U shapes (train / val):", tuple(train_full.shape), tuple(val_full.shape))


Full-U shapes (train / val): (791, 2, 3, 2, 2) (319, 2, 3, 2, 2)



## Build two learners: χ=1 vs χ=2 (temporal bond)

We keep Kraus legs small (2 at input/output) and vertical bonds small (2) so computation is fast.

learned parametrized model with χ=1 vs χ=2 capacity to test presence/absence of memory.


In [6]:

from ptnt.tn.pepo import create_PT_PEPO_guess, expand_initial_guess_

def make_grid(Q, T, chi_temporal=1, kraus_out_in=(2,2), chi_vertical=2):
    # Seed tiny PEPO
    pepo = create_PT_PEPO_guess(
        T, Q,
        [1]*T,
        [[1]*max(Q-1,0) for _ in range(T+1)],
        [[1]+[1]*(T-1)+[1] for _ in range(Q)]
    )
    grid = qu.tensor.tensor_2d.TensorNetwork2DFlat.from_TN(
        pepo, site_tag_id="q{}_I{}", Ly=T+1, Lx=Q, y_tag_id="ROWq{}", x_tag_id="COL{}"
    )
    # Expand capacity
    kout, kin = kraus_out_in
    K_lists = [[kout] + [1]*(T-1) + [kin] for _ in range(Q)]
    horiz = [[chi_temporal]*(T+1) for _ in range(Q)]
    vert  = [[chi_vertical]*max(Q-1,0) for _ in range(T+1)]
    expand_initial_guess_(grid, K_lists, horiz, vert, rand_strength=0.05, squeeze=True)
    grid.squeeze_()
    return grid, grid.Lx, grid.Ly

grid_chi1, Lx1, Ly1 = make_grid(Q, T, chi_temporal=1)
grid_chi2, Lx2, Ly2 = make_grid(Q, T, chi_temporal=2)
print("Grids built. (Lx, Ly):", (Lx1, Ly1), (Lx2, Ly2))


Grids built. (Lx, Ly): (2, 3) (2, 3)



## Fit by maximum likelihood (+causality) and evaluate
this is the “fit the parametrized model” step, and “compare outputs” with formal metrics.

- Loss = cross-entropy between data probabilities and model predictions.
- Add a small causality penalty `kappa`.
- Use `optimizer="adam"` and greedy contraction for speed.

We report:
- **Validation cross-entropy** (compare to **data-entropy** floor).
- **Per-circuit Hellinger fidelity** (median and IQR).


In [7]:

from ptnt.tn.optimize import TNOptimizer
from ptnt.tn.fit import compute_likelihood, causality_keys_to_op_arrays, compute_probabilities
from ptnt.utilities import hellinger_fidelity

def run_fit(grid, Lx, Ly, train_full, val_full, train_vec, val_vec, epochs=2, batch=256, kappa=1e-3):
    train_v = np.array(train_vec, dtype=float); train_v[train_v < 1e-12] = 1e-12
    val_v   = np.array(val_vec,   dtype=float); val_v[val_v   < 1e-12]   = 1e-12

    iterations = int(2 * epochs * len(train_v) / batch)

    optmzr = TNOptimizer(
        grid,
        loss_fn=compute_likelihood,
        causality_fn=causality_keys_to_op_arrays,
        causality_key_size=64,
        training_data=train_v,
        training_sequences=train_full,
        Lx=Lx, Ly=Ly,
        validation_data=list(val_v),
        validation_sequences=val_full,
        batch_size=batch,
        loss_constants={},
        loss_kwargs={"kappa": kappa, "opt": "greedy", "X_decomp": False},
        autodiff_backend="jax",
        optimizer="adam",
        progbar=True,
    )
    _ = optmzr.optimize(iterations)
    best = optmzr.best_val_mpo

    # Predictions and metrics
    pred = compute_probabilities(best, val_full, X_decomp=False, opt="greedy")
    pred = sum(val_v) * pred / sum(pred)

    Qbits = 2**Q
    fids = []
    for i in range(len(val_vec) // Qbits):
        p = np.array(pred[Qbits*i:Qbits*(i+1)]); p = p / p.sum()
        a = np.array(val_v[Qbits*i:Qbits*(i+1)])
        fids.append(hellinger_fidelity(p, a))

    # Epoch-sampled losses (per-epoch last batch)
    nB = optmzr._nBatches
    epoch_losses = [float(optmzr.losses[nB-1 + nB*i]) for i in range(int(optmzr._n / nB))]
    epoch_val_losses = [float(optmzr.val_losses[nB-1 + nB*i]) for i in range(int(optmzr._n / nB))]

    return {
        "epoch_losses": epoch_losses,
        "epoch_val_losses": epoch_val_losses,
        "val_median_fid": float(np.median(fids)),
        "val_iqr_fid": float(np.quantile(fids, 0.75) - np.quantile(fids, 0.25)),
        "val_fids": [float(x) for x in fids],
    }

print("Fitting χ=1...")
res1 = run_fit(grid_chi1, Lx1, Ly1, train_full, val_full, train_vec, val_vec)
print("Fitting χ=2...")
res2 = run_fit(grid_chi2, Lx2, Ly2, train_full, val_full, train_vec, val_vec)
print("Done.")


Fitting χ=1...


+0.2826579 [best loss: +0.2796150] [best val: +0.2518657; (9)]: : 13it [00:03,  3.42it/s]                                                             


Fitting χ=2...


  0%|                                                                                                                          | 0/12 [00:00<?, ?it/s]/home/fazeleh/.local/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
+0.2806771 [best loss: +0.2806771] [best val: +0.2478900; (12)]: : 13it [00:04,  3.12it/s]                                                            


Done.


## Compare to entropy floor & declare PASS/FAIL

In [14]:

# Summarize vs floors and declare PASS/FAIL
def entropy_floor(vec):
    v = np.array(vec, dtype=float)
    v[v < 1e-12] = 1e-12
    return - (1/len(v)) * v @ np.log(v + 1e-18)

v_floor = entropy_floor(val_vec)
ce_chi1 = res1["epoch_val_losses"][-1]
ce_chi2 = res2["epoch_val_losses"][-1]

print(f"Validation floor (entropy): {v_floor:.6f}")
print(f"χ=1  val CE: {ce_chi1:.6f} | median fid: {res1['val_median_fid']:.4f} (IQR {res1['val_iqr_fid']:.4f})")
print(f"χ=2  val CE: {ce_chi2:.6f} | median fid: {res2['val_median_fid']:.4f} (IQR {res2['val_iqr_fid']:.4f})")

if SCENARIO == "baseline":
    # Expect χ=1 close to floor; χ=2 no material improvement
    delta = ce_chi1 - v_floor
    improv = ce_chi1 - ce_chi2
    verdict = ("PASS" if delta < 0.05 and improv < 0.02 else "WARN")
    print(f"[Baseline] Δ(χ=1,floor)={delta:.4f}, improvement χ=2={improv:.4f} → {verdict}")
else:
    # Expect χ=2 beats χ=1 materially
    improv = ce_chi1 - ce_chi2
    verdict = ("PASS" if improv > 0.02 else "WARN")
    print(f"[Memory] improvement χ=2={improv:.4f} → {verdict}")


Validation floor (entropy): 0.235933
χ=1  val CE: 0.252725 | median fid: 0.9994 (IQR 0.2633)
χ=2  val CE: 0.273349 | median fid: 0.9986 (IQR 0.2658)
[Baseline] Δ(χ=1,floor)=0.0168, improvement χ=2=-0.0206 → PASS


- We generated synthetic data (oracle) with known physics (memory present/absent)
- We trained a parameterized model (LPDO) with explicit capacity control (χ).
- We tested on fresh hold-out circuits (new experiments).
- We compared model vs oracle quantitatively (CE vs entropy floor, Hellinger).
- We declared PASS/FAIL with explicit thresholds.
- This is a complete, provable sanity check that our code learns what it should and only when appropriate

the result for the baseline (memory-free) scenario is exactly what we would expect from a correct implementation. our learner with χ = 1 (no temporal memory) achieves a validation cross-entropy only Δ≈0.017 nats above the validation data-entropy floor (0.2527 vs 0.2359), and increasing χ to 2 does not help (CE worsens by ~0.0206). The median Hellinger fidelity ≈ 0.999 on fresh circuits is also as high as it should be in a well-posed baseline. That is a clean PASS for the baseline sanity test and strong evidence that our shadow preprocessing, PEPO→LPDO build, JAX-based likelihood/gradients, and evaluation logic are behaving correctly.

In [29]:

from qiskit_aer import Aer
import numpy as np, quimb as qu
import matplotlib.pyplot as plt

from ptnt.circuits.templates import base_PT_circ_template
from ptnt.circuits.noise_models import create_env_IA, make_coherent_depol_noise_model
from ptnt.circuits.utils import bind_ordered

from ptnt.preprocess.shadow import (
    clifford_param_dict, validation_param_dict, shadow_results_to_data_vec,
    shadow_seqs_to_op_array,
    clifford_measurements_vT, clifford_unitaries_vT,
    val_measurements_vT, val_unitaries_vT,
)

SCENARIO = "memory"   # "baseline" or "memory"

if SCENARIO == "baseline":
    Q, T = 2, 2
    env = create_env_IA(0.0, 0.0, 0.0)
    noise_model = make_coherent_depol_noise_model(0.001, 0.02)
    shots_char, shots_val = 1024, 4096
    N_train, N_val = 240, 80
else:
    Q, T = 2, 4
    env = create_env_IA(0.0, 0.0, 0.35)
    noise_model = None
    shots_char, shots_val = 2048, 32768 
    N_train, N_val = 600, 200

backend = Aer.get_backend("aer_simulator")
shell = base_PT_circ_template(Q, T, backend, basis_gates=None, template="dd_clifford", env_IA=env)

print("Scenario:", SCENARIO, "| Q,T =", (Q, T))
print(shell)


Scenario: memory | Q,T = (2, 4)
              ┌─────────┐          ┌──────────┐                              »
q_0: ─────────┤ Ry(π/4) ├──────────┤0         ├──────────────────────────────»
     ┌────────┴─────────┴─────────┐│  Unitary │┌────────────────────────────┐»
q_1: ┤ U(t0_q0_x,t0_q0_y,t0_q0_z) ├┤1         ├┤ U(t1_q0_x,t1_q0_y,t1_q0_z) ├»
     ├────────────────────────────┤└──────────┘└────────────────────────────┘»
q_2: ┤ U(t0_q1_x,t0_q1_y,t0_q1_z) ├──────────────────────────────────────────»
     └────────────────────────────┘                                          »
c: 2/════════════════════════════════════════════════════════════════════════»
                                                                             »
«     ┌──────────┐         ┌──────────┐                                       »
«q_0: ┤0         ├─────────┤0         ├───────────────────────────────────────»
«     │          │         │  Unitary │         ┌────────────────────────────┐»
«q_1: ┤  Unitary 

In [30]:

def reverse_seq_list(seq_list):
    out = []
    for seq in seq_list:
        tmp = []
        for Tseq in seq:
            tmp.append([o for o in reversed(Tseq)])
        tmp.reverse()
        out.append(tmp)
    return out

def build_batch(template, N, table, T, Q):
    circs, seqs = [], []
    for _ in range(N):
        idx = np.random.randint(0, len(table), size=(T+1, Q))
        seqs.append(idx.T)
        params = np.array([table[i] for i in idx.ravel()])
        circs.append(bind_ordered(template, params.ravel()))
    return circs, seqs

def entropy_floor(prob_vec):
    v = np.array(prob_vec, dtype=float)
    v[v < 1e-12] = 1e-12
    return - (1/len(v)) * v @ np.log(v + 1e-18)

# Generate and simulate
train_circs, train_seqs = build_batch(shell, N_train, clifford_param_dict, T, Q)
val_circs,   val_seqs   = build_batch(shell, N_val,   validation_param_dict, T, Q)

job_t = backend.run(train_circs, shots=shots_char, noise_model=noise_model)
job_v = backend.run(val_circs,   shots=shots_val,   noise_model=noise_model)

train_counts = job_t.result().get_counts()
val_counts   = job_v.result().get_counts()

train_vec, train_keys = shadow_results_to_data_vec(train_counts, shots_char, Q)
val_vec,   val_keys   = shadow_results_to_data_vec(val_counts,   shots_val,   Q)

print("train / val items:", len(train_vec), len(val_vec))
print("data-entropy (train / val):", entropy_floor(train_vec), entropy_floor(val_vec))


train / val items: 2358 800
data-entropy (train / val): 0.2670807534306066 0.2619333641906337


In [31]:

# Convert sequences -> operator arrays (Full-U view)
train_full = shadow_seqs_to_op_array(reverse_seq_list(train_seqs), train_keys, clifford_measurements_vT, clifford_unitaries_vT)
val_full   = shadow_seqs_to_op_array(reverse_seq_list(val_seqs),   val_keys,   val_measurements_vT,   val_unitaries_vT)

print("Full-U shapes (train / val):", tuple(train_full.shape), tuple(val_full.shape))


Full-U shapes (train / val): (2358, 2, 5, 2, 2) (800, 2, 5, 2, 2)


In [35]:

from ptnt.tn.pepo import create_PT_PEPO_guess, expand_initial_guess_

def make_grid(Q, T, chi_temporal=1, kraus_out_in=(2,2), chi_vertical=2):
    # Seed tiny PEPO
    pepo = create_PT_PEPO_guess(
        T, Q,
        [1]*T,
        [[1]*max(Q-1,0) for _ in range(T+1)],
        [[1]+[1]*(T-1)+[1] for _ in range(Q)]
    )
    grid = qu.tensor.tensor_2d.TensorNetwork2DFlat.from_TN(
        pepo, site_tag_id="q{}_I{}", Ly=T+1, Lx=Q, y_tag_id="ROWq{}", x_tag_id="COL{}"
    )
    # Expand capacity
    kout, kin = kraus_out_in
    K_lists = [[kout] + [1]*(T-1) + [kin] for _ in range(Q)]
    horiz = [[chi_temporal]*(T+1) for _ in range(Q)]
    vert  = [[chi_vertical]*max(Q-1,0) for _ in range(T+1)]
    expand_initial_guess_(grid, K_lists, horiz, vert, rand_strength=0.05, squeeze=True)
    grid.squeeze_()
    return grid, grid.Lx, grid.Ly

grid_chi1, Lx1, Ly1 = make_grid(Q, T, chi_temporal=1)
grid_chi2, Lx2, Ly2 = make_grid(Q, T, chi_temporal=3)
print("Grids built. (Lx, Ly):", (Lx1, Ly1), (Lx2, Ly2))


Grids built. (Lx, Ly): (2, 5) (2, 5)


In [38]:

from ptnt.tn.optimize import TNOptimizer
from ptnt.tn.fit import compute_likelihood, causality_keys_to_op_arrays, compute_probabilities
from ptnt.utilities import hellinger_fidelity

def run_fit(grid, Lx, Ly, train_full, val_full, train_vec, val_vec, epochs=2, batch=256, kappa=1e-3):
    train_v = np.array(train_vec, dtype=float); train_v[train_v < 1e-12] = 1e-12
    val_v   = np.array(val_vec,   dtype=float); val_v[val_v   < 1e-12]   = 1e-12

    iterations = int(2 * epochs * len(train_v) / batch)

    optmzr = TNOptimizer(
        grid,
        loss_fn=compute_likelihood,
        causality_fn=causality_keys_to_op_arrays,
        causality_key_size=64,
        training_data=train_v,
        training_sequences=train_full,
        Lx=Lx, Ly=Ly,
        validation_data=list(val_v),
        validation_sequences=val_full,
        batch_size=batch,
        loss_constants={},
        loss_kwargs={"kappa": kappa, "opt": "greedy", "X_decomp": False},
        autodiff_backend="jax",
        optimizer="adam",
        progbar=True,
    )
    _ = optmzr.optimize(iterations)
    best = optmzr.best_val_mpo

    # Predictions and metrics
    pred = compute_probabilities(best, val_full, X_decomp=False, opt="greedy")
    pred = sum(val_v) * pred / sum(pred)

    Qbits = 2**Q
    fids = []
    for i in range(len(val_vec) // Qbits):
        p = np.array(pred[Qbits*i:Qbits*(i+1)]); p = p / p.sum()
        a = np.array(val_v[Qbits*i:Qbits*(i+1)])
        fids.append(hellinger_fidelity(p, a))

    # Epoch-sampled losses (per-epoch last batch)
    nB = optmzr._nBatches
    epoch_losses = [float(optmzr.losses[nB-1 + nB*i]) for i in range(int(optmzr._n / nB))]
    epoch_val_losses = [float(optmzr.val_losses[nB-1 + nB*i]) for i in range(int(optmzr._n / nB))]

    return {
        "epoch_losses": epoch_losses,
        "epoch_val_losses": epoch_val_losses,
        "val_median_fid": float(np.median(fids)),
        "val_iqr_fid": float(np.quantile(fids, 0.75) - np.quantile(fids, 0.25)),
        "val_fids": [float(x) for x in fids],
    }

print("Fitting χ=1...")
res1 = run_fit(grid_chi1, Lx1, Ly1, train_full, val_full, train_vec, val_vec,
               epochs=4, batch=256, kappa=3e-4)
print("Fitting χ=2...")

res2 = run_fit(grid_chi2, Lx2, Ly2, train_full, val_full, train_vec, val_vec,
               epochs=4, batch=256, kappa=3e-4)

print("Done.")


Fitting χ=1...


+0.2573271 [best loss: +0.2565579] [best val: +0.2678060; (70)]: : 74it [00:08,  9.13it/s]                                                            


Fitting χ=2...


+0.2735255 [best loss: +0.2580358] [best val: +0.2709035; (44)]: : 74it [00:10,  7.18it/s]                                                            


Done.


In [39]:

# Summarize vs floors and declare PASS/FAIL
def entropy_floor(vec):
    v = np.array(vec, dtype=float)
    v[v < 1e-12] = 1e-12
    return - (1/len(v)) * v @ np.log(v + 1e-18)

v_floor = entropy_floor(val_vec)
ce_chi1 = res1["epoch_val_losses"][-1]
ce_chi2 = res2["epoch_val_losses"][-1]

print(f"Validation floor (entropy): {v_floor:.6f}")
print(f"χ=1  val CE: {ce_chi1:.6f} | median fid: {res1['val_median_fid']:.4f} (IQR {res1['val_iqr_fid']:.4f})")
print(f"χ=2  val CE: {ce_chi2:.6f} | median fid: {res2['val_median_fid']:.4f} (IQR {res2['val_iqr_fid']:.4f})")

if SCENARIO == "baseline":
    # Expect χ=1 close to floor; χ=2 no material improvement
    delta = ce_chi1 - v_floor
    improv = ce_chi1 - ce_chi2
    verdict = ("PASS" if delta < 0.05 and improv < 0.02 else "WARN")
    print(f"[Baseline] Δ(χ=1,floor)={delta:.4f}, improvement χ=2={improv:.4f} → {verdict}")
else:
    # Expect χ=2 beats χ=1 materially
    improv = ce_chi1 - ce_chi2
    verdict = ("PASS" if improv > 0.02 else "WARN")
    print(f"[Memory] improvement χ=2={improv:.4f} → {verdict}")


Validation floor (entropy): 0.261933
χ=1  val CE: 0.311290 | median fid: 0.9944 (IQR 0.0084)
χ=2  val CE: 0.273546 | median fid: 0.9954 (IQR 0.0077)
[Memory] improvement χ=2=0.0377 → PASS
